In [27]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from collections import defaultdict
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
import copy


from sklearn.metrics import classification_report, accuracy_score

In [53]:
bad = open('sents_with_mistakes.txt.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt.txt', encoding='utf8').read().splitlines()

In [29]:
print(bad[2])
print(true[2])

Пояним эту мысль.
Поясним эту мысль


In [54]:
def prepare_sents(sents):
    tokenized = []
    for sent in sents:
        sent = sent.lower()
        tokens = sent.split()
        tokens = [re.sub('(^\W+|\W+$)', '', token) for token in tokens if (set(token)-punct)]
        tokenized.append(list(tokens))
    return tokenized

In [55]:
bad_tokens = prepare_sents(bad)
true_tokens = prepare_sents(true)

In [56]:
print(bad_tokens[1])

['опофеозом', 'дня', 'для', 'меня', 'сегодня', 'стала', 'фраза', 'услышанная', 'в', 'новостях']


In [57]:
import gzip
import csv

In [32]:
corpus = open('corpus_5000.txt', 'w', encoding = 'utf-8')
with gzip.open('lenta-ru-news.csv.gz', 'rt', encoding='utf-8') as archive:
    reader = csv.reader(archive, delimiter=',', quotechar='"')
    for i, line in enumerate(reader):
        if i < 5000: 
            corpus.write(line[2].replace('\xa0', ' ') + '\n')

In [40]:
def normalize(text):
    normalized_text = [(word.strip(punctuation)) for word in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text

In [41]:
import string
import nltk
from nltk import sent_tokenize
nltk.download('punkt')
punctuation = string.punctuation+ "«»—…“”"
punct = set(punctuation)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lizan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [42]:
corpus = []
for text in open('corpus_5000.txt', encoding = 'utf-8').read().splitlines():
    sents = sent_tokenize(text)
    norm_sents = [normalize(sent) for sent in sents]
    corpus += norm_sents

In [36]:
def clear_from_multiplied_letters(text):  #большое количество повторяющихся букв превращается в две повторяющиеся буквы
    if isinstance(text, list):            
        for i in text:               
            i = clear_from_multiplied_letters(i) 
    elif isinstance(text, str):
        symbols_to_delete = []
        for i in range(len(text)-2):
              if text[i] == text[i+1] and text[i] == text[i+2]:
                symbols_to_delete.append(i)
        if len(symbols_to_delete) > 0:
            textlist = list(text)
            for i in range(len(symbols_to_delete)):
                textlist.pop(-i-1)       #иду от конца к началу, чтобы не сдвигались индексы
                text = ''.join(textlist)
    return text

In [69]:
print(clear_from_multiplied_letters('хорошоооооо'))

хороооо


In [70]:
print(clear_from_multiplied_letters('хорошооо'))

хорошоо


In [ ]:
#Это очень странно, почему-то убирается только одна из разноженных букв. Простите, не успела это исправить.
#В таком виде функция всё же чуть-чуть улучшает ситуацию, поэтому применю

In [71]:
bad_multyclean = clear_from_multiplied_letters(bad_tokens)

In [73]:
def del_letter(word, n):
    word_L = word[:n]
    word_R = word[n:]
    word_ufterdel = word_L + word_R[1:]
    return word_ufterdel

In [77]:
def corpus_del(corpus): #корпус должен быть представлен как список токеноd
    corpusdict = defaultdict(set)
    for word in corpus:
        for n in range(len(word)):
            word1del = del_letter(word, n)
            corpusdict[word1del].add(word)
            if len(word) >= 2:
                 for n2 in range(n):
                    word2del = del_letter(word1del, n2)
                    corpusdict[word2del].add(word)
    return(corpusdict)

In [78]:
corpus_list = []
for sent in corpus:
    for word in sent:
        corpus_list.append(word)       

In [79]:
corpusdict = corpus_del(corpus_list)

In [81]:
print(corpusdict['кока'])

{'склока', 'кнопка', 'кошкам', 'конкау', 'комика', 'конька', 'кошка', 'койках', 'козака', 'кокаин', 'кокату'}


In [83]:
def symspell2dels(text, corpus, corpusdict):
    corrections = {}
    for word in text:
        if word not in corpus:
            correct = set()
            delword = []
            delword.append(word)
            for n in range(len(word)):
                word1del = del_letter(word, n)
                delword.append(word1del)
                for n2 in range(n):
                    word2del = del_letter(word1del, n)
                    delword.append(word2del)     
            for seq in delword:
                if seq in corpus:
                    correct.append(seq)
                if seq in corpusdict:
                    correct.add(optiondict[seq])
        corrections[word] = correct
    return corrections  

In [85]:
bad_list = []
for i in bad_multyclean:
    for j in bad:
        bad_list.append(j)

In [ ]:
corrections = symspell2dels(bad_list, corpus, corpusdict)

In [ ]:
#судя по времени выполнения предыдущей строки, удалять по 2 буквы было стратегической ошибкой

In [ ]:
def choose_corrections_distance(corrections):
    corretions_1option = copy.deepcopy(corrections)
    for word in corrections_1 option:
        if len(correction_1option[word]) > 1:
            correction_1option[word]= get_closest_match_with_metric(word, correction_1option[word], 1, textdistance.damerau_levenshtein)
    return corrections_1option

In [ ]:
corrections_1option = choose_corrections_distance(corrections)

In [ ]:
corrected_using_dist = copy.deepcopy(bad)
for sentence in corrected_using_dist:
    for n in len(sentence):
        word = sentence[n]
        if word in corrections_1option:
            sentence[n] = sentence[word]

# Оценим работу спелчекера, выбирающего одно из наиболее близких слов

In [ ]:
def metrics(true, bad, corrected):
    metrics_dict = {}
    metrics_dict[total_mistaken] = 0
    metrics_dict[really_corrected] = 0
    metrics_dict[not_corrected] = 0
    metrics_dict[broken] = 0
    for i in range(len(true)):
        for j in range(len(true[i])):
            true_word = true[i][j]
            bad_word = bad[i][j]
            corrected_word = corrected[i][j]
            if true_word != bad_word:
                if true_word == corrected_word:
                    metrics_dict[really_corrected] += 1
                else:
                    metrics_dict[total_mistaken] += 1
                    metrics_dict[not_corrected] += 1
            else:
                if true_word != corrected_word:
                    metrics_dict[total_mistaken] += 1
                    metrics_dict[broken] += 1
    return(metrics_dict)       

# Будем выбирать слово в соответствии с частотностью триграм

In [ ]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [ ]:
bigrams = Counter()
trigrams = Counter()

corpus_forNgtams = copy.deepcopy(corpus)

for sentence in corpus_forNgrams:
    sentence_forNgram = [<START>, <START>]
    for word in sentence:
        sentence_forNgram.append(word)
    bigrams.update(ngrammer(sentence_forNgram))
    trigrams.update(ngrammer(sentence_forNgram, 3))

In [ ]:
def trigram_correct(text, corrections, bigrams, bigrams):
    corrected_text = copy.deepcopy(text)
    for n_sent in range(len(text)):
        for i in range(2, len(sentence)):
            if word in corrections:
                wordset = corrections[word]
                if len() == 1:
                    corrected_text[n_sent][i] = wordset.pop()
                else:
                    corrected_prev1 = corrected_text[n_sent][i-1]  #брать одно слово из исправленного текста, а одно из изначального, бесполезно:
                    corrected_prev2 = corrected_text[n_sent][i-2]  #если какое-то из предыдущих двух слов было исправлено, значит, его нет в корпусе,
                    text_prev1 = text[n_sent][i-1]                #в таком случае вероятность триграмы и биграмы с изначальным словом равна нулю
                    text_prev2 = text[n_sent][i-2]
                    possible_prev = set(corrected_prev2 + ' ' + corrected_prev1, text_prev2 + ' ' + text_prev1)
                    max_p = 0              
                    max_p_words = []
                    for option in wordset:
                        for bigram in possible_prev:
                            trigram = bigram + ' ' + option
                            try:
                                p = trigrams[trigram]/bigrams[bigram]
                            except:
                                p = 0
                            if p == max_p:
                                max_p_words.append(option)
                            elif p > max_p:
                                max_p_words = []
                            max_p = p
                if len(max_p_words) = 1:
                     corrected_text[n_sent][i] = max_p_words[0]
                elif len(max_p_words) > 1: #если есть несколько равновероятных кандидатов (например, если предшествующей биграмы в корпусе не было),
                    chosen = None         #выбираем самое близкое
                    for cand in max_p_words: 
                        chosen= get_closest_match_with_metric(word, max_p_words, 1, textdistance.damerau_levenshtein)
                    corrected_text[n_sent][i] = chosen   
    return corrected_text